In [1]:
import pandas
import pyperf
import json

In [2]:
def get_benchmark_suite(path):
    with open(path, 'r') as file:
        return pyperf.BenchmarkSuite.load(file)

In [3]:
def get_data_frame(benchmark_suite):
    rows = []
    for benchmark in benchmark_suite:
        name = benchmark.get_name()

        for value in benchmark.get_values():
            rows.append(dict(name=name, value=value))
    
    return pandas.DataFrame(rows)

In [4]:
names = ['baseline', 'v4.1.2', 'sub', 'key opt', 'tss opt', 'preshed opt', 'map opt', 'time opt', 'subopt']

# load data frames
data_frames = {name: get_data_frame(get_benchmark_suite(f'./{name}.json')) for name in names}
# aggregate and merge data frames
data_frame = pandas.concat({name: data_frame.groupby('name').mean() * 1e3 for name, data_frame in data_frames.items()}).unstack(0).droplevel(0, axis='columns')
# sort columns by
if True:  # average factor
    sort_values = data_frame.iloc[:, 1:].apply(lambda series: series / data_frame.iloc[:, 0]).dropna().mean()
else:  # average rank
    ranks = data_frame.iloc[:, 1:].rank(axis='columns')
    sort_values = ranks.apply(lambda series: (series - 1) / (ranks.max(axis='columns') - 1)).dropna().mean()
data_frame = data_frame.loc[:, ['baseline'] + list(sort_values.index[sort_values.argsort()[::-1]])]
# convert times to factors
data_frame.iloc[:, 1:] = data_frame.iloc[:, 1:].apply(lambda series: series / data_frame.iloc[:, 0])
# sort by last (fastest)
data_frame = data_frame.sort_values([names[-1], 'name'], ascending=[False, True])
# highlight best in row
best = data_frame.iloc[:, 1:].min(axis='columns')
data_frame.iloc[:, 1:] = data_frame.iloc[:, 1:].apply(lambda series: pandas.Series([f'**{number:.2f}**' if is_best else f'{number:.2f}' for number, is_best in zip(series, series == best)], index=series.index))

def _(data_frame):
    return data_frame.to_string(float_format=lambda float: f'{float:0.2f}')
    
def _(data_frame):
    return data_frame.to_markdown(floatfmt='0.2f')

print(_(data_frame))

sort_values.sort_values()

| name                       |   baseline |   v4.1.2 |    sub |   preshed opt |   key opt | time opt   | tss opt   | map opt   | subopt    |
|:---------------------------|-----------:|---------:|-------:|--------------:|----------:|:-----------|:----------|:----------|:----------|
| generators                 |      57.18 |    52.70 |  73.47 |         56.27 |     56.43 | 53.65      | 53.36     | 52.26     | **48.58** |
| deltablue                  |       4.46 |    36.44 |  38.27 |         30.90 |     30.04 | 28.67      | 28.51     | 28.87     | **25.98** |
| richards                   |      54.57 |    34.22 |  33.56 |         26.44 |     25.54 | 24.36      | **23.47** | 23.68     | 23.72     |
| richards_super             |      66.02 |    31.38 |  31.43 |         24.68 |     23.29 | 22.82      | 21.88     | 22.03     | **21.61** |
| raytrace                   |     331.44 |    26.21 |  29.19 |         24.02 |     22.78 | 21.95      | 21.72     | 21.25     | **20.72** |
| scimark_sor

subopt         6.469236
map opt        6.803570
tss opt        6.811526
time opt       6.994645
key opt        7.311419
preshed opt    7.437348
sub            9.407143
v4.1.2         9.573182
dtype: float64